In [1]:
import  pandas as pd
import numpy as np
import selenium

In [ ]:
df = pd.read_csv('kursach_08.12.24.csv', sep=',')
df

,Unnamed: 0,Event ID,Event Date,Event Text,Story ID,City,District,Province,Latitude,Longitude,Year
0,7,37542842,2022-10-19,Make statement,56633384,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2022
1,8,37542843,2022-10-19,Threaten,56633384,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2022
2,10,37543059,2022-10-19,Impose state of emergency or martial law,56635208,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2022
3,14,37540464,2022-10-20,Accuse,56627735,Cherkassy,NaN,Lipetskaya Oblast',52.6959,38.7262,2022
4,15,37540279,2022-10-20,Make a visit,56627791,Ryazan',NaN,Ryazanskaya Oblast',54.6269,39.6916,2022
...,...,...,...,...,...,...,...,...,...,...,...
275095,1199562,37532072,2023-01-03,fight with small arms and light weapons,56607018,Rakovka,NaN,Smolenskaya Oblast',54.0909,32.8770,2023
275096,1199567,37532083,2023-01-03,fight with small arms and light weapons,56607164,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2023
275097,1199571,37531846,2023-01-03,Use conventional military force,56607320,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2023
275098,1199573,37531983,2023-01-03,Use conventional military force,56607471,Lugansk,NaN,Novosibirskaya Oblast',53.6220,77.5991,2023


In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [5]:
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome()


In [9]:
# driver = webdriver.Chrome()
options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# options.page_load_strategy = 'eager'
driver.get("https://yandex.ru/maps/")



In [10]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from fake_useragent import UserAgent
# ua = UserAgent()


In [11]:
def get_municipality(latitude, longitude):
    try:

        # Находим поле поиска

        search_box = driver.find_element(By.CSS_SELECTOR, "input.input__control")
        search_box.clear()

        # Вводим координаты
        coordinates = f"{latitude}, {longitude}"
        search_box.send_keys(coordinates)
        print(Keys.RETURN)
        search_box.send_keys(Keys.RETURN)
        time.sleep(0.1)  # Ждём загрузки результата


        # Извлекаем информацию о муниципалитете
        # Считываем первый результат в правом блоке
        search_box = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.toponym-card-title-view__description'))
            )

        municipality = search_box.text
        print(municipality)
        search_box = driver.find_element(By.XPATH, "//button[contains(@aria-label, 'Закрыть')]") 
        search_box.click()

        return municipality
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

In [13]:
# from tqdm import tqdm

# Создаём столбец для результатов
df["municipality"] = None
# Парсим каждую координату
for idx, row in df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    municipality = get_municipality(latitude, longitude)
    df.at[idx, "municipality"] = municipality
    print(f"{idx}/{275100}")
    if idx%5502==0 and idx!=0:
        time.sleep(60*3)
    

# Сохраняем результаты в файл
df.head()


1-й Михайловский проезд, Михайловский район, Рязань
0/275100

Гайский муниципальный округ, Оренбургская область
1/275100

Гайский муниципальный округ, Оренбургская область
2/275100

Комсомольская улица, 19, село Черкассы, Елецкий район, Липецкая область, 399768
3/275100

1-й Михайловский проезд, Михайловский район, Рязань
4/275100

1-й Михайловский проезд, Михайловский район, Рязань
5/275100

1-й Михайловский проезд, Михайловский район, Рязань
6/275100

1-й Михайловский проезд, Михайловский район, Рязань
7/275100

Гайский муниципальный округ, Оренбургская область
8/275100

Гайский муниципальный округ, Оренбургская область
9/275100

Дворцовая площадь, Санкт-Петербург
10/275100

Дворцовая площадь, Санкт-Петербург
11/275100

проспект Богдана Хмельницкого, 80, Белгород
12/275100

Муезерское городское поселение, Республика Карелия
13/275100

деревня Терехово, Пановское сельское поселение, Палехский район, Ивановская область
14/275100

деревня Терехово, Пановское сельское по

KeyboardInterrupt: 

In [ ]:
df.to_csv('all_fin.csv')